In [ ]:
#regress out
import os
import ssm
import scipy.io
import numpy as np
import pandas as pd
from mne.time_frequency import psd_array_multitaper
import matplotlib.pyplot as plt
import umap
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LinearRegression
def sdfgen(spike,window,overlap,totallength):
    timelength=int((totallength-window)/(window-overlap))
    timestamp=np.arange(0,timelength)*(window-overlap)
    sdf=np.zeros((len(spike),len(timestamp)))
    for i in range(len(spike)):
        for j in spike[i]:
            sdf[i]+=np.exp(-(j-timestamp)*(j-timestamp)/(2*(window-overlap)*(window-overlap)))
    return sdf
def psdcal(lfp1,lfp2,fs,window,overlap,totallength,startfreq,endfreq):
    lfp1=lfp1.T
    lfp2=lfp2.T
    timelength=int((totallength-window)/(window-overlap))
    freqlength=int(200*window*2)+1
    result1=np.zeros((timelength,freqlength))
    result2=np.zeros((timelength,freqlength))
    freq=[]
    lfp1=np.mean(lfp1,axis=0)
    lfp2=np.mean(lfp2,axis=0)
    for j in range(timelength):
        x=np.zeros(int(2*window*fs))
        x[:int(window*fs)]+=lfp1[int(j*(window-overlap)*fs):int(j*(window-overlap)*fs)+int(fs*window)]
        f,Pxx=psd_array_multitaper(x,sfreq=fs,bandwidth=3,adaptive=True,normalization='full',verbose=False)
        result1[j]=f[:400*window+1]
        freq=Pxx
    for j in range(timelength):
        x=np.zeros(int(2*window*fs))
        x[:int(window*fs)]+=lfp2[int(j*(window-overlap)*fs):int(j*(window-overlap)*fs)+int(fs*window)]
        f,Pxx=psd_array_multitaper(x,sfreq=fs,bandwidth=3,adaptive=True,normalization='full',verbose=False)
        result2[j]=f[:400*window+1]
        freq=Pxx
    result1=np.sum(result1[:,startfreq*window*2:endfreq*window*2+1],axis=1,keepdims=True)
    result2=np.sum(result2[:,startfreq*window*2:endfreq*window*2+1],axis=1,keepdims=True)
    return result1,result2
def synchronycal(lfp1,lfp2):
    return np.corrcoef(lfp1,lfp2)[0][1]
answer=[[],[],[]]
originalanswer=[[],[],[]]
nonsocialanswer=[[],[],[]]
randanswer=[[],[],[]]
nonsocialanswer=[[],[],[]]
overlap=1.5
window=2
binlength=window-overlap
folder_path='path_to_matlab_struct_files'
selfsocial_cell="path_to_SocialNeuron_auROC_self.csv"
othersocial_cell="path_to_SocialNeuron_auROC_other.csv"
for file in os.listdir(folder_path):
    if file.endswith('.mat') and not file.endswith('_LFP.mat'):
        matfilepath=os.path.join(folder_path,file)
        lfpfilepath=os.path.join(folder_path,file[:-4]+'_LFP.mat')
        if not os.path.exists(lfpfilepath):
            continue
        Kfile=scipy.io.loadmat(matfilepath)['K']
        LFPfile=scipy.io.loadmat(lfpfilepath)
        selfsocialcell=pd.read_csv(selfsocial_cell)
        selfsocialcell=selfsocialcell.values.tolist()
        othersocialcell=pd.read_csv(othersocial_cell)
        othersocialcell=othersocialcell.values.tolist()
        samplingfreq=float(LFPfile['SamplingFreq'])
        rawlfp1=LFPfile['raw_signal_final'][0][0]
        rawlfp2=LFPfile['raw_signal_final'][0][1]
        if (len(Kfile['Event'][0][0][0][3])>1):
            basetime=Kfile['Event'][0][0][0][3][0]/1000000
            endtime=Kfile['Event'][0][0][0][3][1]/1000000
        else:
            basetime=Kfile['Event'][0][0][0][3][0][0]/1000000
            endtime=Kfile['Event'][0][0][0][3][0][1]/1000000
        m1ID=matfilepath[-23:-4]+Kfile['ID'][0][0][0][0][0]
        m2ID=matfilepath[-23:-4]+Kfile['ID'][0][0][0][1][0]        
        interactionindex=0
        if ('WT' in m1ID) and ('KO' in m2ID):
            interactionindex=1
        if ('WT' in m1ID) and ('WT' in m2ID):
            interactionindex=0
        if ('KO' in m1ID) and ('KO' in m2ID):
            interactionindex=2
        if ('KO' in m1ID) and ('WT' in m2ID):
            interactionindex=1
        if ('KO' in m1ID):
            m1=1
        else:
            m1=0
        if ('KO' in m2ID):
            m2=1
        else:
            m2=0        
        if 'SPKm1' in Kfile.dtype.names:
            # print(Kfile['SPKm1'][0][0])
            spike1raw=Kfile['SPKm1'][0][0]-basetime
            spike2raw=Kfile['SPKm2'][0][0]-basetime
        else:
            continue
        date=matfilepath[-23:-4]
        m1name=Kfile['ID'][0][0][0][0][0]
        m2name=Kfile['ID'][0][0][0][1][0]
        totalspike1raw=[spike1raw[i][0] for i in range(len(spike1raw))]
        totalspike2raw=[spike2raw[i][0] for i in range(len(spike2raw))]
        spike1rawsocial=[spike1raw[i][0] for i in range(len(spike1raw)) if ([f"'{date}'",f"'{m1name}'",i+1] in selfsocialcell) or ([f"'{date}'",f"'{m1name}'",i+1] in othersocialcell)]
        spike2rawsocial=[spike2raw[i][0] for i in range(len(spike2raw)) if ([f"'{date}'",f"'{m2name}'",i+1] in selfsocialcell) or ([f"'{date}'",f"'{m2name}'",i+1] in othersocialcell)]
        spike1nonsocial=[spike1raw[i][0] for i in range(len(spike1raw)) if not (([f"'{date}'",f"'{m1name}'",i+1] in selfsocialcell) or ([f"'{date}'",f"'{m1name}'",i+1] in othersocialcell))]
        spike2nonsocial=[spike2raw[i][0] for i in range(len(spike2raw)) if not (([f"'{date}'",f"'{m2name}'",i+1] in selfsocialcell) or ([f"'{date}'",f"'{m2name}'",i+1] in othersocialcell))]
        totallength=int(int(np.min([float(endtime-basetime),len(rawlfp1)/samplingfreq,len(rawlfp2)/samplingfreq,600])/binlength)*binlength)           
        spike1=sdfgen(spike1rawsocial,window,overlap,totallength)
        spike2=sdfgen(spike2rawsocial,window,overlap,totallength)
        if len(spike1)+len(spike2)>0:
            model=LinearRegression()
            lfp1,lfp2=psdcal(rawlfp1,rawlfp2,samplingfreq,window,overlap,totallength,4,12)
            if (len(spike1)>0) & (len(spike1raw)>len(spike1)):
                model.fit(spike1.T,np.log(lfp1))
                outlfp1=np.exp(np.log(lfp1)-model.predict(spike1.T)).flatten()
            else:
                outlfp1=lfp1.flatten()
            model=LinearRegression()
            if (len(spike2)>0)&(len(spike2raw)>len(spike2)):
                model.fit(spike2.T,np.log(lfp2))
                outlfp2=np.exp(np.log(lfp2)-model.predict(spike2.T)).flatten()
            else:
                outlfp2=lfp2.flatten()
            answer[interactionindex].append(synchronycal(outlfp1,outlfp2))
            tempsync=[]
            nonsocialtempsync=[]
            for seed in range(100):
                np.random.seed(seed)
                model=LinearRegression()
                if (len(spike1)>0) & (len(totalspike1raw)>=len(spike1)):
                    randspike1=sdfgen([totalspike1raw[index] for index in np.random.choice(range(len(totalspike1raw)),len(spike1rawsocial),replace=False)],window,overlap,totallength)
                    
                    model.fit(randspike1.T,np.log(lfp1))
                    randoutlfp1=np.exp(np.log(lfp1)-model.predict(randspike1.T)).flatten()
                else:
                    randoutlfp1=lfp1.flatten()
                model=LinearRegression()
                if (len(spike2)>0)&(len(totalspike2raw)>=len(spike2)):
                    randspike2=sdfgen([totalspike2raw[index] for index in np.random.choice(range(len(totalspike2raw)),len(spike2rawsocial),replace=False)],window,overlap,totallength)    
                    model.fit(randspike2.T,np.log(lfp2))
                    randoutlfp2=np.exp(np.log(lfp2)-model.predict(randspike2.T)).flatten()
                else:
                    randoutlfp2=lfp2.flatten()
                model=LinearRegression()
                if (len(spike1)>0) & (len(spike1nonsocial)>=len(spike1)):
                    nonsocialrandspike1=sdfgen([spike1nonsocial[index] for index in np.random.choice(range(len(spike1nonsocial)),len(spike1rawsocial),replace=False)],window,overlap,totallength)
                    
                    model.fit(nonsocialrandspike1.T,np.log(lfp1))
                    nonsocialrandoutlfp1=np.exp(np.log(lfp1)-model.predict(nonsocialrandspike1.T)).flatten()
                else:
                    nonsocialrandoutlfp1=lfp1.flatten()
                model=LinearRegression()
                if (len(spike2)>0)&(len(spike2nonsocial)>=len(spike2)):
                    nonsocialrandspike2=sdfgen([spike2nonsocial[index] for index in np.random.choice(range(len(spike2nonsocial)),len(spike2rawsocial),replace=False)],window,overlap,totallength)    
                    model.fit(nonsocialrandspike2.T,np.log(lfp2))
                    nonsocialrandoutlfp2=np.exp(np.log(lfp2)-model.predict(nonsocialrandspike2.T)).flatten()
                else:
                    nonsocialrandoutlfp2=lfp2.flatten()                    
                tempsync.append(synchronycal(randoutlfp1,randoutlfp2)) 
                nonsocialtempsync.append(synchronycal(nonsocialrandoutlfp1,nonsocialrandoutlfp2))   
            randanswer[interactionindex].append(np.mean(tempsync))
            nonsocialanswer[interactionindex].append(np.mean(nonsocialtempsync))
            originalanswer[interactionindex].append(synchronycal(lfp1.flatten(),lfp2.flatten()))
pd.DataFrame(answer).to_csv('regressoutanswer.csv')
pd.DataFrame(nonsocialanswer).to_csv('regressoutnonsocialanswer.csv')
pd.DataFrame(randanswer).to_csv('regressouttotalrandanswer.csv')   
pd.DataFrame(originalanswer).to_csv('originalanswer.csv')     


2023-07-04_14-20-5104KO
0
0
12
1


C:\Users\Anjin\AppData\Local\Temp\ipykernel_27872\4182123137.py:189: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  samplingfreq=float(LFPfile['SamplingFreq'])
C:\Users\Anjin\AppData\Local\Temp\ipykernel_27872\4182123137.py:233: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  totallength=int(int(np.min([float(endtime-basetime),len(rawlfp1)/samplingfreq,len(rawlfp2)/samplingfreq,600])/binlength)*binlength)
